## Model used: Naive Bayes and Decision tree
### Data used: Data without PCA
### Methods used: RandomizedSearchCV
### Accuracy with NaiveBayes: train:42%
### Accuracy with DecisionTree: train:95% , test:95%

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

In [3]:
df = pd.read_csv("/home/aban/somu/review-2/processed_data.csv")

In [4]:
df.describe()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,37,38,39,40,41,42,43,44,45,46
count,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,...,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07
mean,7.569746e+06,8.920319e+03,8.770264e+00,1.226366e+07,2.358934e+01,6.334113e+00,9.783180e+02,2.019014e+02,1.113722e+01,5.057569e+01,...,5.057569e+01,8.829068e+03,8.740443e+03,1.798149e+01,1.736567e+05,8.675735e+04,2.635969e+05,1.160456e+05,4.690261e+06,1.702228e-01
std,4.370395e+06,1.869813e+04,4.929919e+00,3.096609e+07,1.520511e+03,1.640832e+02,6.301678e+04,3.042774e+02,2.428807e+01,6.063933e+01,...,6.063933e+01,1.626557e+04,2.066946e+04,7.715197e+00,2.510512e+06,1.515901e+06,3.324653e+06,2.118789e+06,1.582677e+07,3.758284e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.784873e+06,5.300000e+01,6.000000e+00,5.140000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.000000e+00,-1.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,7.569746e+06,8.000000e+01,6.000000e+00,2.269900e+04,2.000000e+00,1.000000e+00,4.300000e+01,4.100000e+01,0.000000e+00,3.600000e+01,...,3.600000e+01,9.800000e+02,-1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.135462e+07,3.389000e+03,1.700000e+01,3.050693e+06,5.000000e+00,4.000000e+00,4.910000e+02,3.120000e+02,3.000000e+01,6.800000e+01,...,6.800000e+01,8.192000e+03,2.370000e+02,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.513949e+07,6.553500e+04,1.700000e+01,1.200000e+08,3.096290e+05,1.231180e+05,1.443918e+08,6.444000e+04,1.460000e+03,1.652931e+04,...,1.652931e+04,6.553500e+04,6.553500e+04,5.600000e+01,1.140000e+08,7.523241e+07,1.140000e+08,1.140000e+08,1.200000e+08,1.000000e+00


In [5]:
df.drop(columns='Unnamed: 0',inplace=True)

In [7]:
df.columns = ['Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Mean', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Pkt Len Min', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt',
       'URG Flag Cnt', 'Down/Up Ratio', 'Fwd Seg Size Avg',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Seg Size Min',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Min','Label']

In [8]:
df.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Bwd Pkt Len Max,...,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Min,Label
0,53.0,17.0,2291.0,1.0,1.0,49.0,49.0,49.0,49.000000,164.0,...,49.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,80.0,6.0,5785762.0,3.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,8192.0,8192.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
2,80.0,6.0,13513.0,3.0,4.0,287.0,287.0,0.0,95.666667,935.0,...,95.666667,65535.0,219.0,20.0,0.0,0.0,0.0,0.0,0.0,1.0
3,80.0,6.0,1610.0,3.0,4.0,308.0,308.0,0.0,102.666667,935.0,...,102.666667,65535.0,219.0,20.0,0.0,0.0,0.0,0.0,0.0,1.0
4,53.0,17.0,1115.0,1.0,1.0,36.0,36.0,36.0,36.000000,73.0,...,36.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
for column in df:
 if df[column].dtype == 'float64':
    df[column]=pd.to_numeric(df[column], downcast='float')
 if df[column].dtype == 'int64':
    df[column]=pd.to_numeric(df[column], downcast='integer')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15139493 entries, 0 to 15139492
Data columns (total 47 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           float32
 1   Protocol           float32
 2   Flow Duration      float64
 3   Tot Fwd Pkts       float32
 4   Tot Bwd Pkts       float32
 5   TotLen Fwd Pkts    float64
 6   Fwd Pkt Len Max    float32
 7   Fwd Pkt Len Min    float32
 8   Fwd Pkt Len Mean   float64
 9   Bwd Pkt Len Max    float32
 10  Bwd Pkt Len Min    float32
 11  Bwd Pkt Len Mean   float64
 12  Flow Byts/s        float64
 13  Flow Pkts/s        float64
 14  Flow IAT Mean      float64
 15  Flow IAT Std       float64
 16  Flow IAT Max       float64
 17  Flow IAT Min       float64
 18  Bwd IAT Tot        float64
 19  Bwd IAT Mean       float64
 20  Bwd IAT Std        float64
 21  Bwd IAT Max        float64
 22  Bwd IAT Min        float64
 23  Fwd PSH Flags      float32
 24  Fwd URG Flags      float32
 25  Fwd Pkts/s      

In [11]:
train_X = df.drop(columns="Label",axis=1)

In [12]:
train_Y = df['Label']

In [13]:
del(df)

In [15]:
scaler= MinMaxScaler();
# X_train_sc = pd.DataFrame(scaler.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

In [16]:
# del(train_X)

In [17]:
# var = np.logspace(0,-9, num=10)
# print(var)
# np.random.seed(999)
# params = {'var_smoothing': var}

# x_clf_2 = GaussianNB()
# random_clf_2 = RandomizedSearchCV(x_clf_2, param_distributions=params, scoring='f1', verbose=10, cv=2)

[1.e+00 1.e-01 1.e-02 1.e-03 1.e-04 1.e-05 1.e-06 1.e-07 1.e-08 1.e-09]


In [18]:
# random_clf_2.fit(train_X,train_Y)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2; 1/10] START var_smoothing=1.0..........................................
[CV 1/2; 1/10] END ...........var_smoothing=1.0;, score=0.000 total time=  10.9s
[CV 2/2; 1/10] START var_smoothing=1.0..........................................
[CV 2/2; 1/10] END ...........var_smoothing=1.0;, score=0.000 total time=  11.0s
[CV 1/2; 2/10] START var_smoothing=0.1..........................................
[CV 1/2; 2/10] END ...........var_smoothing=0.1;, score=0.305 total time=  11.1s
[CV 2/2; 2/10] START var_smoothing=0.1..........................................
[CV 2/2; 2/10] END ...........var_smoothing=0.1;, score=0.305 total time=  11.2s
[CV 1/2; 3/10] START var_smoothing=0.01.........................................
[CV 1/2; 3/10] END ..........var_smoothing=0.01;, score=0.307 total time=  11.2s
[CV 2/2; 3/10] START var_smoothing=0.01.........................................
[CV 2/2; 3/10] END ..........var_smoothing=0.01;

RandomizedSearchCV(cv=2, estimator=GaussianNB(),
                   param_distributions={'var_smoothing': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09])},
                   scoring='f1', verbose=10)

In [19]:
# random_clf_2.best_score_

0.41920898906119114

In [20]:
model2 = GaussianNB(var_smoothing=random_clf_2.best_params_['var_smoothing'])
model2.fit(train_X,train_Y)

GaussianNB()

In [21]:
ypred = model2.predict(train_X)
f1score = f1_score(train_Y, ypred)
print('f1_score for train:',f1score)

f1_score for train: 0.4196348622743802


In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
from scipy.stats import randint

params = {'max_depth' : [3,5,8,None],
         'max_features': ['auto','log2',None],
         }
x_clf_5 = DecisionTreeClassifier()

random_clf_5 = RandomizedSearchCV(x_clf_5, param_distributions=params, scoring='f1', verbose=1, cv=2, n_jobs=3)


In [16]:
random_clf_5.fit(train_X,train_Y)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/home/aban/somu/env/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/home/aban/somu/env/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/home/aban/somu/env/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/home/aban/somu/env/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  war

RandomizedSearchCV(cv=2, estimator=DecisionTreeClassifier(), n_jobs=3,
                   param_distributions={'max_depth': [3, 5, 8, None],
                                        'max_features': ['auto', 'log2', None]},
                   scoring='f1', verbose=1)

In [17]:
random_clf_5.best_score_

0.960487527426306

In [18]:
random_clf_5.best_params_

{'max_features': None, 'max_depth': 8}

In [19]:
modelD = DecisionTreeClassifier(max_depth=8)

In [20]:
modelD.fit(train_X,train_Y)

DecisionTreeClassifier(max_depth=8)

In [21]:
ypred = modelD.predict(train_X)
f1_sc = f1_score(train_Y,ypred)
print(f1_sc)

0.9599663557494829


In [36]:
test_data = pd.read_csv("/home/aban/somu/test_prepro.csv")

In [37]:
test_data.head()

,Unnamed: 0,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Down/Up Ratio,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Min
0,0,443.0,6.0,1653.0,3.0,0.0,77.0,46.0,0.0,25.666667,...,0.0,25.666667,255.0,-1.0,20.0,0.0,0.0,0.0,0.0,0.0
1,1,443.0,6.0,713.0,3.0,0.0,77.0,46.0,0.0,25.666667,...,0.0,25.666667,258.0,-1.0,20.0,0.0,0.0,0.0,0.0,0.0
2,2,443.0,6.0,20623981.0,8.0,7.0,740.0,388.0,0.0,92.500000,...,0.0,92.500000,8192.0,7483.0,20.0,231049.0,0.0,231049.0,231049.0,20392473.0
3,3,53.0,17.0,1321.0,1.0,1.0,33.0,33.0,33.0,33.000000,...,1.0,33.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0
4,4,53.0,17.0,745.0,1.0,1.0,47.0,47.0,47.0,47.000000,...,1.0,47.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0


In [38]:
test_data=test_data.drop(columns="Unnamed: 0",axis=1)
test_data.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Bwd Pkt Len Max,...,Down/Up Ratio,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Min
0,443.0,6.0,1653.0,3.0,0.0,77.0,46.0,0.0,25.666667,0.0,...,0.0,25.666667,255.0,-1.0,20.0,0.0,0.0,0.0,0.0,0.0
1,443.0,6.0,713.0,3.0,0.0,77.0,46.0,0.0,25.666667,0.0,...,0.0,25.666667,258.0,-1.0,20.0,0.0,0.0,0.0,0.0,0.0
2,443.0,6.0,20623981.0,8.0,7.0,740.0,388.0,0.0,92.500000,1460.0,...,0.0,92.500000,8192.0,7483.0,20.0,231049.0,0.0,231049.0,231049.0,20392473.0
3,53.0,17.0,1321.0,1.0,1.0,33.0,33.0,33.0,33.000000,87.0,...,1.0,33.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0
4,53.0,17.0,745.0,1.0,1.0,47.0,47.0,47.0,47.000000,113.0,...,1.0,47.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0


In [39]:
y_pred_on_test = modelD.predict(test_data)

In [40]:
type(y_pred_on_test)

numpy.ndarray

In [41]:
dtree_pred_test = pd.DataFrame(y_pred_on_test)

In [42]:
dtree_pred_test = dtree_pred_test[0].astype(np.int8)
dtree_pred_test

0         0
1         0
2         0
3         0
4         0
         ..
999995    0
999996    0
999997    0
999998    0
999999    0
Name: 0, Length: 1000000, dtype: int8

In [43]:
dtree_pred_test.head()

0    0
1    0
2    0
3    0
4    0
Name: 0, dtype: int8

In [34]:
dtree_pred_test.to_csv("./dtree_pred_test.csv")